In [3]:
### This script looks at all files
### 1) Checks deleted files for md5 related fields and clears them
###                         for qc metric and changes status of qc metric object to deleted
###                         clears the qc metric field
###                         for workflows and deletes all of them
### 2) Checks other files for workflows and deleted old workflows
###                                     and deleted problematic ones (status or rev)
### takes around 20 min

from core.utils import Tibanna
from core import ff_utils
from datetime import datetime

# set enviroment and key/connection
env = 'fourfront-webprod'
tibanna = Tibanna(env=env)

tibanna.ff_keys['default']['server'] = 'https://data.4dnucleome.org'
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)
tibanna.ff_keys['default']['server'] = 'https://data.4dnucleome.org'


In [11]:
print 'started at', datetime.utcnow()

# what kind of files should be searched for worflow run inputs, use url compatible naming

# accepted workflows
# workflow name, accepted revision numbers (0 if none), accetable run time (hours)
workflow_details = [
                    ['md5', ['0'], 12],
                    ['fastqc-0-11-4-1', ['0', '1'], 12],
#                     ['hi-c-processing-parta-juicer', ['25','26'], 150],
#                     ['hi-c-processing-partb', ['31', '34', '38'],150],
#                     ['hi-c-processing-partc', ['3', '8'], 150],
#                     ['add-hic-normvector-to-mcool', ['3'], 150],
#                     ['extract-mcool-normvector-for-juicebox', ['100'], 150],
#                     ['extract-mcool-normvector-for-juicebox-1', ['1'], 150],
#                     ['bwa-mem', ['0'], 150],
#                     ['pairsam-parse-sort',['0'],150],
#                     ['pairsam-merge',['0'],150],
#                     ['pairsam-markasdup',['0'],150],
#                     ['pairsam-filter',['0'],150],
#                     ['addfragtopairs',['0'],150],
#                     ['pairs-patch',['0'],150],
#                     ['hi-c-processing-partb set',['0'],150],
#                     ['hi-c-processing-partb exp',['0'],150],
#                     ['hi-c-processing-partc set',['0'],150],
#                     ['hi-c-processing-partc exp',['0'],150],
    
                    ['bwa-mem 0.2.5', ['0'],50],
                    ['pairsqc-single 0.2.5', ['0'],50],
                    ['hi-c-processing-bam 0.2.5', ['0'],50],  
                    ['hi-c-processing-pairs 0.2.5', ['0'],50],
                    ['hi-c-processing-pairs-nore 0.2.5', ['0'],50],
                    ['hi-c-processing-pairs-nonorm 0.2.5', ['0'],50],
                    ['hi-c-processing-pairs 0.2.5', ['0'],50],
                   ]
workflow_names = [i[0] for i in workflow_details]

deleted_wfr_no = 0
files_with_deleted_wfr = 0


file_url = '/search/?status=released&status=released%20to%20project&type=FileFastq'
files = ff_utils.get_metadata(file_url , connection=ff)['@graph']

rel_p = 0
rel = 0

print len(files), 'files in the system'

counter = 0
for a_file in files:
    counter += 1
    if counter % 100 == 0:
        print counter, rel, rel_p
    wfr_report = []
    f_status = a_file['status']
    wfrs = a_file.get('workflow_run_inputs')
    if not wfrs:
        continue
    for i in wfrs:
        wfr_meta = ff_utils.get_metadata(i , connection=ff)
        if wfr_meta['run_status'] != 'complete':
            print('problem', a_file['accession'], wfr_meta['uuid'])
        wfr_status = wfr_meta['status']
        
        if f_status != wfr_status:
            
            if f_status == 'released':
                rel += 1
            elif f_status == 'released to project':
                rel_p += 1
            patch_data = {'status': f_status} 
            ff_utils.patch_metadata(patch_data, obj_id=i ,connection=ff)

print('rel', rel)
print('rel_p', rel_p)
          
print 'finished at', datetime.utcnow()

started at 2018-05-14 21:17:26.356771
2943 files in the system
100 0 0
200 0 28
300 150 67
400 150 267
500 150 467
600 174 684
700 188 879
800 445 879
900 556 1009
1000 571 1199
1100 571 1399
1200 571 1599
1300 571 1799
1400 649 2003
1500 879 2003
1600 1039 2003
1700 1291 2003
1800 1561 2003
1900 1861 2003
2000 2161 2003
2100 2461 2003
2200 2761 2003
2300 2980 2003
2400 3180 2003
2500 3332 2003
2600 3519 2003


2700 3849 2003
2800 4009 2003
2900 4041 2003
('rel', 4061)
('rel_p', 2003)
finished at 2018-05-14 22:07:14.582784
